In [2]:
#!/usr/bin/env python3
"""
hourly_sumo_pipeline.py
-----------------------
Chạy 24 phiên mô phỏng SUMO, mỗi giờ một phiên, với khả năng resume.
Mỗi artefact (trips, routes, TLS cycle, TLS offset, state, outputs) được
lưu trữ và nếu đã tồn tại thì bỏ qua bước đó.

Sửa lỗi liên quan đến TLS khi load state:
  - Gom tất cả tlLogic từ 00..hour thành một file merged, để không có duplicate.
  - Lưu state đúng vào thời điểm kết thúc mỗi giờ (3600, 7200, …).
"""

from __future__ import annotations

import os
import sys
import subprocess
from pathlib import Path
from typing import Optional, Dict
import xml.etree.ElementTree as ET

# ---------------------------------------------------------------------------
#  Project helpers và cấu hình
# ---------------------------------------------------------------------------

sys.path.append("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/src")

from simulationHelpers import od2tripsForAllOLD  # noqa: E402
from config import TLSCYCLEADAPTATION_PY, TLSCOORDINATOR_PY  # noqa: E402

# ---------------------------------------------------------------------------
#  Constants
# ---------------------------------------------------------------------------

THREADS = min(1, os.cpu_count() or 1)
os.environ["OMP_NUM_THREADS"] = str(THREADS)

NET_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/connected-network.net.xml"
TAZ_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/net-creation/310525-AMP-test-func/taz.xml"
VTYPES_XML = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/additional-files/vtypes/eidm-1.xml"
ODS_DIR = "/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/data/processed-data/total_scenario/old-form-all"

SIM_DIR = Path("/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/03-06-hourly-scenario-2")
TRIPS_DIR = SIM_DIR / "trips"
ROUTES_DIR = SIM_DIR / "routes"
TLS_DIR = SIM_DIR / "tls"
OUTPUTS_DIR = SIM_DIR / "outputs"
STATES_DIR = SIM_DIR / "states"
LOGS_DIR = SIM_DIR / "logs"

for p in (TRIPS_DIR, ROUTES_DIR, TLS_DIR, OUTPUTS_DIR, STATES_DIR, LOGS_DIR):
    p.mkdir(parents=True, exist_ok=True)

HOUR = 3600
STEP_LENGTH = 0.5
SEED = "2005"
TIME_TO_TELEPORT = "120"
PROGRAM_ID = "a"

# ---------------------------------------------------------------------------
#  Helper functions
# ---------------------------------------------------------------------------

def outputs_complete(tag: str) -> bool:
    outdir = OUTPUTS_DIR / f"hour_{tag}"
    files = [
        outdir / "summary.xml",
        outdir / "tripinfo.xml",
        outdir / "statistics.xml",
        outdir / "vehRoutes.xml",
        outdir / "laneChanges.xml",
        outdir / "collisions.xml",
    ]
    if not all(f.exists() and f.stat().st_size > 0 for f in files):
        return False

    try:
        content = (outdir / "statistics.xml").read_text(encoding="utf-8")
    except Exception:
        return False

    remainder = content.split("-->", maxsplit=1)[-1]
    return "<performance" in remainder

def state_path(end_time: int) -> Path:
    return STATES_DIR / f"state_{end_time:08d}.xml.gz"


def merge_tls_cycles(hour: int) -> Path:
    """
    Gom tất cả tl_cycle_jj.add.xml từ j=0..hour thành một file merged, sao cho
    mỗi <tlLogic id="X"> chỉ xuất hiện lần cuối (giờ muộn nhất) trong merged file.
    Lưu vào TLS_DIR/tls_cycle_merged_HH.add.xml.
    """
    merged_path = TLS_DIR / f"tls_cycle_merged_{hour:02d}.add.xml"
    if merged_path.exists() and merged_path.stat().st_size > 0:
        return merged_path

    # Dict id -> element (của tlLogic) cuối cùng
    tl_dict: Dict[str, ET.Element] = {}
    for j in range(hour + 1):
        cycle_file = TLS_DIR / f"tls_cycle_{j:02d}.add.xml"
        if not cycle_file.exists():
            continue
        try:
            tree = ET.parse(cycle_file)
        except Exception:
            continue
        root = tree.getroot()
        for tl in root.findall(".//tlLogic"):
            tl_id = tl.attrib.get("id")
            if tl_id:
                # ghi đè (giữ tlLogic từ giờ muộn hơn)
                tl_dict[tl_id] = tl

    # Tạo root <additional> chứa tất cả tlLogic
    additional = ET.Element("additional")
    for tl in tl_dict.values():
        # append a deep copy để tránh sửa gốc
        additional.append(ET.fromstring(ET.tostring(tl)))

    tree = ET.ElementTree(additional)
    tree.write(merged_path, encoding="utf-8", xml_declaration=True)
    return merged_path


def tls_files_up_to(hour: int) -> str:
    """
    Trả chuỗi comma-separated gồm:
    - File tls_cycle_merged_HH.add.xml (merge từ 00..hour).
    - Tất cả tls_coord_jj.add.xml từ j=0..hour.
    """
    merged_cycle = merge_tls_cycles(hour)
    coord_files = [str(TLS_DIR / f"tls_coord_{j:02d}.add.xml")
                   for j in range(hour + 1)
                   if (TLS_DIR / f"tls_coord_{j:02d}.add.xml").exists()]
    return ",".join([str(merged_cycle), *coord_files])

# ---------------------------------------------------------------------------
#  Generate trip files nếu missing
# ---------------------------------------------------------------------------

trip_files = sorted(TRIPS_DIR.glob("*.xml"))
if len(trip_files) < 24:
    print("[INFO] generating trip files …")
    trip_FILES = sorted(od2tripsForAllOLD(TAZ_XML, TRIPS_DIR, ODS_DIR, "car_eidm"))
    trip_files = trip_FILES

if len(trip_files) != 24:
    sys.exit("[ERROR] need exactly 24 trip files (one per hour)")

# ---------------------------------------------------------------------------
#  Determine resume point
# ---------------------------------------------------------------------------

prev_state: Optional[Path] = None
start_hour = 0
for idx in reversed(range(24)):
    if outputs_complete(f"{idx:02d}"):
        start_hour = idx + 1
        prev_state = state_path((idx + 1) * HOUR)
        break
print(f"Resume from hour {start_hour:02d} (prev_state={prev_state})")

# ---------------------------------------------------------------------------
#  Main loop
# ---------------------------------------------------------------------------

for hour_idx in range(start_hour, 24):
    tag = f"{hour_idx:02d}"
    begin = hour_idx * HOUR
    end = (hour_idx + 1) * HOUR

    # Nếu outputs đã đầy đủ, skip
    if outputs_complete(tag):
        print(f"[H{tag}] outputs already complete – skip")
        prev_state = state_path(end)
        continue

    trips_xml = Path(trip_files[hour_idx])

    # -------------------------------------------------------------------
    #  1) Tạo hoặc reuse route file
    # -------------------------------------------------------------------

    route_xml = ROUTES_DIR / f"route_{tag}.xml"
    if not route_xml.exists() or route_xml.stat().st_size == 0:
        print(f"[H{tag}] duarouter …")
        duarouter_log = LOGS_DIR / f"duarouter_{tag}.log"
        cmd = [
            "duarouter", "-n", NET_XML, "-r", str(trips_xml), "-a", VTYPES_XML,
            "-o", str(route_xml), "--log", str(duarouter_log),
            "--exit-times", "--named-routes", "--route-length", "--write-costs",
            "--routing-threads", "8", "--ignore-errors", "--seed", SEED,
        ]
        subprocess.run(cmd, check=True)
    else:
        print(f"[H{tag}] route ready – skip duarouter")

    # -------------------------------------------------------------------
    #  2) Tạo hoặc reuse TLS cycle
    # -------------------------------------------------------------------

    tls_cycle = TLS_DIR / f"tls_cycle_{tag}.add.xml"
    if not tls_cycle.exists() or tls_cycle.stat().st_size == 0:
        print(f"[H{tag}] tlsCycleAdaptation …")
        adapt_log = LOGS_DIR / f"tls_adapt_{tag}.log"
        cmd = [
            "python3", TLSCYCLEADAPTATION_PY,
            "-n", NET_XML, "-r", str(route_xml), "-b", str(begin),
            "-o", str(tls_cycle),
            "--min-cycle", "40", "--max-cycle", "120",
            "--yellow-time", "3", "-p", PROGRAM_ID, "--verbose",
        ]
        with adapt_log.open("w") as log_f:
            subprocess.run(cmd, stdout=log_f, stderr=subprocess.STDOUT, check=True)
    else:
        print(f"[H{tag}] tls_cycle ready – skip adaptation")

    # -------------------------------------------------------------------
    #  3) Tạo hoặc reuse TLS coord
    # -------------------------------------------------------------------

    tls_coord = TLS_DIR / f"tls_coord_{tag}.add.xml"
    if not tls_coord.exists() or tls_coord.stat().st_size == 0:
        print(f"[H{tag}] tlsCoordinator …")
        coord_log = LOGS_DIR / f"tls_coord_{tag}.log"
        cmd = [
            "python3", TLSCOORDINATOR_PY,
            "-n", NET_XML, "-r", str(route_xml),
            "-a", str(tls_cycle), "-o", str(tls_coord),
        ]
        with coord_log.open("w") as log_f:
            subprocess.run(cmd, stdout=log_f, stderr=subprocess.STDOUT, check=True)
    else:
        print(f"[H{tag}] tls_coord ready – skip coordination")

    # -------------------------------------------------------------------
    #  4) Chạy SUMO cho giờ này
    # -------------------------------------------------------------------

    outdir = OUTPUTS_DIR / f"hour_{tag}"
    outdir.mkdir(exist_ok=True)
    summary_xml    = outdir / "summary.xml"
    tripinfo_xml   = outdir / "tripinfo.xml"
    statistics_xml = outdir / "statistics.xml"
    vehroutes_xml  = outdir / "vehRoutes.xml"
    lanechanges_xml= outdir / "laneChanges.xml"
    collision_xml  = outdir / "collisions.xml"
    state_file     = state_path(end)

    virtual_end = end + 1

    sumo_cmd = [
        "sumo", "-n", NET_XML, "-r", str(route_xml),
        "--threads", str(THREADS),
        "--additional-files", tls_files_up_to(hour_idx),
        "--step-length", str(STEP_LENGTH),
        "--begin", str(begin), "--end", str(virtual_end),
        "--summary-output",   str(summary_xml),
        "--tripinfo-output",  str(tripinfo_xml),
        "--statistic-output", str(statistics_xml),
        "--vehroute-output",  str(vehroutes_xml),
        "--lanechange-output",str(lanechanges_xml),
        "--collision-output", str(collision_xml),
        "--time-to-teleport", TIME_TO_TELEPORT,
        "--ignore-junction-blocker", "20",
        "--lateral-resolution", "0.4",
        "--ignore-route-errors",
        "--no-step-log", "--duration-log.statistics",
        "--save-state.times", str(end),         
        "--save-state.files", str( state_file),
        "--xml-validation", "never",
    ]

    if prev_state and prev_state.exists():
        sumo_cmd += ["--load-state", str(prev_state)]

    print(f"[H{tag}] SUMO {begin}→{end} …")
    sim_log = LOGS_DIR / f"sumo_{tag}.log"
    with sim_log.open("w") as log_f:
        subprocess.run(sumo_cmd, stdout=log_f, stderr=subprocess.STDOUT, check=True)
    print(f"[H{tag}] finished (state saved → {state_file})\n")

    prev_state = state_file

print("All 24 hourly simulations finished")


Resume from hour 08 (prev_state=/home/hoai-linh.dao/Works/EVCS/CEREMA-Mini/result/experiments/03-06-hourly-scenario-2/states/state_00028800.xml.gz)
[H08] route ready – skip duarouter
[H08] tls_cycle ready – skip adaptation
[H08] tls_coord ready – skip coordination
[H08] SUMO 28800→32400 …


KeyboardInterrupt: 